In [46]:
import pandas as pd

# Import, lowercase city/state name to attempt join later

# Geonames is ~1:1 City/Place names to Zip Codes
# Schema: http://download.geonames.org/export/zip/readme.txt
col_names = ['country_code',
             'postal_code',
             'place_name',
             'admin_name1',
             'admin_code1',
             'admin_name2',
             'admin_code2',
             'admin_name3',
             'admin_code3',
             'latitude',
             'longitude',
             'accuracy']
geonames = pd.read_table('GeoNamesZip.txt', header=None, names=col_names, dtype={'postal_code': str})
geonames['place_name'] = geonames['place_name'].str.lower()
geonames['admin_code1'] = geonames['admin_code1'].str.lower()
geonames_set = set(geonames['postal_code'])

# Free-zip 1 is 1:1 City to Zip Code
free_zip_db = pd.read_csv('free-zipcode-database-Primary.csv', dtype={'Zipcode': str})
free_zip_db['City'] = free_zip_db['City'].str.lower()
free_zip_db['State'] = free_zip_db['State'].str.lower()
freezip_set = set(free_zip_db['Zipcode'])

# Free-zip all is Many:1 City to Zip Code
free_zip_all = pd.read_csv('free-zipcode-database-all-places.csv', dtype={'Zipcode': str})
free_zip_all['City'] = free_zip_all['City'].str.lower()
free_zip_all['State'] = free_zip_all['State'].str.lower()
freezip_all_set = set(free_zip_all['Zipcode'])

#
noncensus = pd.read_csv('noncensus_zip.csv', dtype={'zip': str})
noncensus['city'] = noncensus['city'].str.lower()
noncensus['state'] = noncensus['state'].str.lower()
noncensus_set = set(noncensus['zip'])

set_lengths = {
    'noncensus_set': len(noncensus_set),
    'free_zip_set': len(freezip_set),
    'free_zip_all_set': len(freezip_all_set),
    'geonames_set': len(geonames_set)
    }
set_lengths

{'free_zip_all_set': 42522,
 'free_zip_set': 42522,
 'geonames_set': 43586,
 'noncensus_set': 43524,
 'simple_set': 28737}

In [43]:
len(geonames), len(free_zip_db), len(free_zip_all), len(noncensus)

(43629, 42522, 81831, 43524, 71815)

In [49]:
# Columns of interest
geo_zip_city_state = geonames[['postal_code', 'place_name', 'admin_code1']]
free_zip_city_state = free_zip_db[['Zipcode', 'City', 'State']]
free_zip_all_city_state = free_zip_all[['Zipcode', 'City', 'State']]
noncensus_city_state = noncensus[['zip', 'city', 'state']]

In [56]:
# free-zip vs geonames
merged_zip = pd.merge(free_zip_all_city_state, geo_zip_city_state, 
                      left_on='Zipcode', right_on='postal_code')
merged_zip_city = pd.merge(free_zip_all_city_state, geo_zip_city_state, 
                           left_on=['Zipcode', 'City'], 
                           right_on=['postal_code', 'place_name'])
merged_zip_state = pd.merge(free_zip_all_city_state, geo_zip_city_state, 
                            left_on=['Zipcode', 'State'], 
                            right_on=['postal_code', 'admin_code1'])
merged_zip_city_state = pd.merge(free_zip_all_city_state, geo_zip_city_state, 
                                 left_on=['Zipcode', 'City', 'State'], 
                                 right_on=['postal_code', 'place_name', 'admin_code1'])
lengths_free_geo = {
    'merged_zip': len(merged_zip),
    'merged_zip_city': len(merged_zip_city),
    'merged_zip_state': len(merged_zip_state),
    'merged_zip_city_state': len(merged_zip_city_state)
}
lengths_free_geo

{'merged_zip': 78300,
 'merged_zip_city': 41404,
 'merged_zip_city_state': 41404,
 'merged_zip_state': 78300}

In [55]:
merged_zip

,Zipcode,City,State,postal_code,place_name,admin_code1
0,07093,monitor,nj,07093,west new york,nj
1,07093,taurus,nj,07093,west new york,nj
2,07093,west new york,nj,07093,west new york,nj
3,07093,guttenberg,nj,07093,west new york,nj
4,07675,westwood,nj,07675,westwood,nj
5,07675,old tappan,nj,07675,westwood,nj
6,07675,river vale,nj,07675,westwood,nj
7,07675,rivervale,nj,07675,westwood,nj
8,07677,woodcliff lake,nj,07677,woodcliff lake,nj
9,07677,westwood,nj,07677,woodcliff lake,nj


In [59]:
# noncensus vs free zip
merged_non_free_zip = pd.merge(free_zip_all_city_state, noncensus_city_state,
                               right_on=['zip'], left_on=['Zipcode'])
merged_non_free_zip_city = pd.merge(free_zip_all_city_state, noncensus_city_state,
                                    right_on=['zip', 'city'], left_on=['Zipcode', 'City'])
merged_non_free_zip_state = pd.merge(free_zip_all_city_state, noncensus_city_state,
                                    right_on=['zip', 'state'], left_on=['Zipcode', 'State'])
merged_non_free_zip_city_state = pd.merge(free_zip_all_city_state, noncensus_city_state,
                                          right_on=['zip', 'city', 'state'], left_on=['Zipcode', 'City', 'State'])
lengths_noncensus_free = {
    'merged_zip': len(merged_non_free_zip),
    'merged_zip_city': len(merged_non_free_zip_city),
    'merged_zip_state': len(merged_non_free_zip_state),
    'merged_zip_city_state': len(merged_non_free_zip_city_state)
}
lengths_noncensus_free

{'merged_zip': 80852,
 'merged_zip_city': 41553,
 'merged_zip_city_state': 41527,
 'merged_zip_state': 80798}

In [64]:
merged_non_free_zip

,Zipcode,City,State,zip,city,state
0,00704,parc parque,pr,00704,aguirre,pr
1,00704,paseo costa del sur,pr,00704,aguirre,pr
2,00704,sect lanausse,pr,00704,aguirre,pr
3,00704,urb eugene rice,pr,00704,aguirre,pr
4,00704,urb gonzalez,pr,00704,aguirre,pr
5,00704,urb la fabrica,pr,00704,aguirre,pr
6,00704,urb monte soria 2,pr,00704,aguirre,pr
7,00704,villas del coqui,pr,00704,aguirre,pr
8,00704,aguirre,pr,00704,aguirre,pr
9,00704,est de trinitaria,pr,00704,aguirre,pr


In [61]:
# What are our state conflicts?
merged_non_free_zip[~merged_non_free_zip['Zipcode'].isin(merged_non_free_zip_state['Zipcode'])]

,Zipcode,City,State,zip,city,state
247,10004,new york,ny,10004,new york,nj
248,10004,bowling green,ny,10004,new york,nj
20960,38041,henning,tn,38041,henning,ar
20961,38041,fort pillow,tn,38041,henning,ar
21583,38063,ripley,tn,38063,ripley,ar
21708,38079,tiptonville,tn,38079,tiptonville,ky
28034,52761,muscatine,ia,52761,muscatine,il
28946,56129,ellsworth,mn,56129,ellsworth,ia
28947,56027,elmore,mn,56027,elmore,ia
29297,55954,mabel,mn,55954,mabel,ia


In [73]:
# Merge all the things
merged_all = pd.merge(merged_non_free_zip, merged_zip,
                      how='left',
                      left_on=['Zipcode', 'City', 'State'], 
                      right_on=['Zipcode', 'City', 'State'])

In [74]:
merged_all

,Zipcode,City,State,zip,city,state,postal_code,place_name,admin_code1
0,00704,parc parque,pr,00704,aguirre,pr,NaN,NaN,NaN
1,00704,paseo costa del sur,pr,00704,aguirre,pr,NaN,NaN,NaN
2,00704,sect lanausse,pr,00704,aguirre,pr,NaN,NaN,NaN
3,00704,urb eugene rice,pr,00704,aguirre,pr,NaN,NaN,NaN
4,00704,urb gonzalez,pr,00704,aguirre,pr,NaN,NaN,NaN
5,00704,urb la fabrica,pr,00704,aguirre,pr,NaN,NaN,NaN
6,00704,urb monte soria 2,pr,00704,aguirre,pr,NaN,NaN,NaN
7,00704,villas del coqui,pr,00704,aguirre,pr,NaN,NaN,NaN
8,00704,aguirre,pr,00704,aguirre,pr,NaN,NaN,NaN
9,00704,est de trinitaria,pr,00704,aguirre,pr,NaN,NaN,NaN


In [75]:
len(merged_all)

80996

In [100]:
# Some renaming
one_zip_col = merged_all[['Zipcode', 'City', 'city', 'place_name', 
                          'State', 'state', 'admin_code1']]
master_zips = one_zip_col.rename(columns={'City': 'City_1', 
                                          'city': 'City_2', 
                                          'place_name': 'City_3',
                                          'State': 'State_1',
                                          'state': 'State_2', 
                                          'admin_code1': 'State_3'})

In [107]:
# Designate rows with matching cities as primary
master_zips['Primary'] = False
def flag_primary(row):
    if row['City_1'] == row['City_2'] == row['City_3']:
        row['Primary'] = True
    return row
mask = master_zips.apply(flag_primary, axis=1)['Primary']
primary_zips = master_zips[mask]

In [108]:
len(primary_zips)

41015